<a href="https://colab.research.google.com/github/zhesun0304/ECON3916/blob/main/Lab6/The%20Architecture%20of%20Bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Lab 6] The Architecture of Bias

In [6]:
# The Danger of Randomness (Manual Split)

import seaborn as sns
import pandas as pd
import numpy as np

# Data Ingestion on the population
df = sns.load_dataset('titanic')
print(f"Total Population: {len(df)}")
print(f"Population Survival Rate: {df['survived'].mean():.4f}")

# Simulating of sampling
# Ensure reproducibility using a seed
np.random.seed(2026)
indices = np.random.permutation(len(df))

Total Population: 891
Population Survival Rate: 0.3838


In [7]:
# Split and Bias Check
# 80/20 Split
split_point = int(0.8 * len(indices))

# Slicing the shuffled indices
train_idx = indices[:split_point]
test_idx = indices[split_point:]

# Creating subsets
train_set = df.iloc[train_idx]
test_set = df.iloc[test_idx]

# Bias Check
train_surv = train_set['survived'].mean()
test_surv = test_set['survived'].mean()
delta = abs(train_surv - test_surv)

print(f"Train Survival Rate: {train_surv:.4f}")
print(f"Test Survival Rate:  {test_surv:.4f}")
print(f"Sampling Bias (Delta): {delta:.4f}")

Train Survival Rate: 0.3736
Test Survival Rate:  0.4246
Sampling Bias (Delta): 0.0510


In [8]:
from sklearn.model_selection import train_test_split

# Stratification (The Fix)

# Stratify by 'pclass' ensures the distribution of classes is identical
X_train, X_test = train_test_split(
    df,
    test_size=0.2,
    random_state=2026,
    stratify=df["pclass"]
)

print("\n--- Stratified Split ---")
print("Train Class Dist:\n", X_train['pclass'].value_counts(normalize=True))
print("Test Class Dist:\n", X_test['pclass'].value_counts(normalize=True))


--- Stratified Split ---
Train Class Dist:
 pclass
3    0.550562
1    0.242978
2    0.206461
Name: proportion, dtype: float64
Test Class Dist:
 pclass
3    0.553073
1    0.240223
2    0.206704
Name: proportion, dtype: float64


In [9]:
from scipy.stats import chisquare

# The SRM Diagnostic (Forensics)
# 1. Observed and expected counts
observed = np.array([450, 550])      # Control, Treatment
expected = np.array([500, 500])      # Planned 50/50 split

# 2. Chi-Square test
chi2_stat, p_value = chisquare(f_obs=observed, f_exp=expected)

print(f"Chi-Square Statistic: {chi2_stat:.4f}")
print(f"P-value: {p_value:.6f}")

# 3. Conclusion
if p_value < 0.01:
    print("CRITICAL FAILURE: Sample Ratio Mismatch (SRM) Detected. Check Load Balancer.")
else:
    print("Variance is within natural limits.")


Chi-Square Statistic: 10.0000
P-value: 0.001565
CRITICAL FAILURE: Sample Ratio Mismatch (SRM) Detected. Check Load Balancer.
